In [1]:
import csv, importlib
import corpus as corpus_class
import categories, filters, vocabulary_builders
from feature_extractors import multinomial_model, tfidf
import numpy as np
from scipy import sparse

In [2]:
cats = categories.categories()

corpus = corpus_class.corpus(cats)
corpus.load("question_train.csv", "question_category_train.csv")

sentence_filters = [filters.punctuation_filter]
word_filters = [filters.small_word_filter, filters.stopword_filter, filters.stemming_filter]

corpus.process(sentence_filters, word_filters, tr_set_size=10000);
print(corpus)

14 categories. 
14417 docuemnts loaded from file. 
processed: True 
	 Training-set, Test-set size: (10000, 4417) 
		 sentence_filters: ['punctuation_filter'] 
		 word_filters: ['small_word_filter', 'stopword_filter', 'stemming_filter'] 
made numeric features: False 



In [3]:
corpus.make_features(-1)
corpus.save()

In [4]:
corpus.X_te.shape

(4417, 11413)

In [83]:
from sklearn.naive_bayes import BernoulliNB
nb_clf = BernoulliNB()
nb_clf.fit(corpus.X_tr, np.array(corpus.y_tr, dtype=int))
nb_clf.score(corpus.X_te, np.array(corpus.y_te, dtype=int))

0.40932759791713835

In [101]:
from sklearn.naive_bayes import MultinomialNB
nb_clf = MultinomialNB(alpha=0.1)
nb_clf.fit(corpus.X_tr, corpus.y_tr)
nb_clf.score(corpus.X_te, corpus.y_te)

0.58093728775186781

In [108]:
from sklearn import tree
tree_clf = tree.DecisionTreeClassifier()
tree_clf = tree_clf.fit(corpus.X_tr, corpus.y_tr)
tree_clf.score(corpus.X_te, corpus.y_te)

0.48019017432646594

In [113]:
from sklearn.ensemble import RandomForestClassifier
forrest_clf = RandomForestClassifier(n_estimators=20,max_features=100)
forrest_clf = forrest_clf.fit(corpus.X_tr, corpus.y_tr)
forrest_clf.score(corpus.X_te, corpus.y_te)

0.51845143762734891

In [5]:
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression(penalty='l1', C=4, solver='liblinear')
lr_clf = lr_clf.fit(corpus.X_tr, corpus.y_tr)
clf = lr_clf
lr_clf.score(corpus.X_te, corpus.y_te)

0.59361557618292959

In [30]:
from sklearn.linear_model import Lasso
lasso_clf = Lasso(alpha=0.001)
lasso_clf = lasso_clf.fit(corpus.X_tr, corpus.y_tr)
lasso_clf.score(corpus.X_te, corpus.y_te)

0.17696540866359312

# Examples

In [60]:
def mk_pred_expl(expl):
    X_expl = corpus.process_example(expl)
    _, J, V = sparse.find(X_expl)
    print(np.matrix([[*corpus.term_space[J]], [*V]]))
    print("->",corpus.cats[ int(clf.predict(X_expl)[0]) ])

In [61]:
expl = ["Ich liebe meinen Mann nicht. Soll ich es ihm sagen?"]
mk_pred_expl(expl)

[['lieb' 'mann' 'sag']
 ['0.53953584567' '0.464431378217' '0.702285245584']]
-> liebe_and_beziehung


In [62]:
expl = ["Um wieviel uhr spielt die deutsche Fussball-Nationalmanschaft heute?"]
mk_pred_expl(expl)

[['fussball' 'wieviel' 'deutsch' 'spielt' 'heut' 'uhr' 'nationalmanschaft']
 ['0.332613019813' '0.231939920504' '0.289425410238' '0.297019202041'
  '0.314676995143' '0.433343795279' '0.613821245703']]
-> freizeit_and_sport


In [63]:
expl = ["Wichsen Titte Fussball tobias spielen"]
mk_pred_expl(expl)

[['fussball' 'spiel' 'titt' 'wichs' 'tobias']
 ['0.327995195145' '0.282926902931' '0.499527764592' '0.559745964134'
  '0.499527764592']]
-> adult
